# Databricks App Compliance Monitor

This notebook monitors and manages Databricks apps that are not on the approved list. Schedule this notebook as often as you want.

**Workflow:**
1. Retrieves all active Databricks apps in the workspace
2. Identifies apps that are not in the approved list
3. Stops unapproved apps and collects creator emails
4. Creates or updates an alert to notify app creators about the compliance issue

**Alert Details:**
- Scheduled to trigger 2 minutes after execution
- Notifies app creators that their app is not approved


In [0]:
# List of approved apps in the organization
approved_apps = [
    "davids-test-app"
]

In [0]:
from databricks.sdk.service.apps import App
from databricks.sdk import WorkspaceClient
from datetime import datetime, timedelta, timezone
import requests

w = WorkspaceClient()

# Get all active apps and extract their names
active_apps = w.apps.list()
active_apps_names = [app.name for app in active_apps]

# Identify apps that are not approved
# unapproved_apps = [app for app in active_apps_names if app not in set(approved_apps)]
unapproved_apps = approved_apps

# Collect creator emails for unapproved apps (optionally stop them)
unapproved_app_creators = []
for app_name in unapproved_apps:
    try:
        w.apps.stop(name=app_name)
    except:
        print(f"Error stopping app {app_name}")
    app_object = w.apps.get(name=app_name)
    unapproved_app_creators.append({"user_email": app_object.creator})

print(f"Found {len(unapproved_apps)} unapproved app(s)")
print(unapproved_app_creators)

In [0]:
# Get Databricks context and authentication
context = dbutils.notebook.entry_point.getDbutils().notebook().getContext()
hostname = context.apiUrl().get()
headers = {"Authorization": f"Bearer {context.apiToken().get()}"}

# Schedule alert to trigger 2 minutes from now
run_time = (datetime.now(timezone.utc) + timedelta(minutes=2)).replace(second=0, microsecond=0)
quartz_cron = f"{run_time.second} {run_time.minute} {run_time.hour} {run_time.day} {run_time.month} ?"

# Define alert configuration
alert_body = {
    "custom_description": "The Databricks App you created is not approved. It has been suspended for now. Please reach out to your Databricks Admin to approve",
    "custom_summary": "Unapproved Databricks App",
    "display_name": "Alert - Unapproved Databricks App",
    "evaluation": {
        "comparison_operator": "LESS_THAN",
        "notification": {
            "notify_on_ok": True,
            "subscriptions": unapproved_app_creators
        },
        "source": {
            "aggregation": "SUM",
            "display": "x",
            "name": "x"
        },
        "threshold": {
            "value": {
                "double_value": 1.25
            }
        }
    },
    "parent_path": "/Workspace/Users/david.hurley@databricks.com",
    "query_text": "SELECT 1 as x",
    "schedule": {
        "pause_status": "UNPAUSED",
        "quartz_cron_schedule": quartz_cron,
        "timezone_id": "UTC"
    },
    "warehouse_id": "383cc3b75ffe2072"
}

# Helper function to make API calls
def make_api_call(method, endpoint, json_body=None):
    url = f"{hostname}/api/2.0/{endpoint}"
    if method == "GET":
        return requests.get(url=url, headers=headers)
    elif method == "POST":
        return requests.post(url=url, headers=headers, json=json_body)
    elif method == "PATCH":
        return requests.patch(url=url, headers=headers, json=json_body)

# Try to create the alert
response = make_api_call("POST", "alerts", alert_body)

# If alert already exists, update it with the latest subscriber list
if response.status_code != 200:
    # Get existing alerts
    response = make_api_call("GET", "alerts")
    response_json = response.json()
    
    # Find the alert ID by display name
    alert_id = next(
        (a["id"] for a in response_json.get("alerts", []) 
         if a.get("display_name") == alert_body["display_name"]), 
        None
    )
    
    # Update the alert
    if alert_id:
        response = make_api_call("PATCH", f"alerts/{alert_id}?update_mask=*", alert_body)
        print(f"Alert updated: {alert_id}")
    else:
        print("Error: Could not find or create alert")
else:
    print("Alert created successfully")